In [1]:
# Jupyter notebook for analyzing DOGAMI data, see Scott Tse's emergence-response notebook at
# https://github.com/hackoregon/emergency-response/blob/analytics/notebooks/census_eda_geo.ipynb
# NOTE: Don't need all of these!
# Import modules included in "kitchen-sink"
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import gdal
import pandas as pd
import psycopg2
import seaborn as sns
import sys
# Import modules NOT included in "kitchen-sink", not sure about osgeo...
from dotenv import load_dotenv, find_dotenv
import geoplot as gplt
from osgeo import osr, ogr 
#import rasterio
from rasterstats import zonal_stats

%matplotlib inline

F.D. Pearce, 04/16/18

Notebook for computing statistics on raster pixel values contained within a geometry (shape) file

In [11]:
# Define ALL parameters in dictionary (convert to json config file!)
params = {
    'raster': {
        'root': './CSZ_M9p0_',
        'names': ['pgv_site', 'PGD_landslide_dry', 'PGD_landslide_wet', 'PGD_liquefaction_wet'],
        'ext': '.tif'
    },
    'geometry': {
        'from_file': {
            'name':'./Data/DisasterNeighborhoods/RLIS_ST_clips_pdx_final.dbf',
            'geom_col': 'geometry'
        }
    },
    'zonal_stats': {
        'layer': 1,
        'stats': ['count', 'min', 'max', 'mean', 'std']
        
    },
    'stats_classification': {
        'stats_to_class': ['min', 'max', 'mean'],
        'pgv_site': {
            'levels': [-9999, 0.1, 1.1, 3.4, 8.1, 16, 31, 60, 116, 9999],
            'level_labels': ['I', 'II-III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X'],
            'class_name': 'Modified Mercalli Intensity',
            'class_tag': 'MMI'
        },
        'PGD_landslide_dry': {
          'levels': [-9999, 0, 10, 30, 100, 9999],
          'level_labels': ['None', 'Low', 'Moderate', 'High', 'Very High'],
          'class_name': 'Landslide Intensity (Dry)',
          'class_tag': 'DI'
        },
        'PGD_landslide_wet': {
          'levels': [-9999, 0, 10, 30, 100, 9999],
          'level_labels': ['None', 'Low', 'Moderate', 'High', 'Very High'],
          'class_name': 'Landslide Intensity (Wet)',
          'class_tag': 'DI'
        },
        'PGD_liquefaction_wet': {
          'levels': [-9999, 0, 10, 30, 100, 9999],
          'level_labels': ['None', 'Low', 'Moderate', 'High', 'Very High'],
          'class_name': 'Liquefaction Intensity (Wet)',
          'class_tag': 'DI'
        }
    },
    'write_csv': {
        'name': "./DisasterNeighborhoods_DogamiRasters_stats.csv"
    }
}

In [3]:
# Functions for obtaining geopandas dataframe containing geometry column from either
# 1) a local file (e.g. .dba file) or 2) interacting with PostGres database
def pgconnect():
    """Establish connection to PostGres database using the parameters specified in .env file.
    First, walk root diretory to find and load .env file w/ PostGres variables defining database, 
    user, host, password, and port variables.
    Then, return connection to database from psycopg2.connect
    """
    try:
        load_dotenv(find_dotenv())
        conn = psycopg2.connect(database=os.environ.get("PG_DATABASE"), user=os.environ.get("PG_USER"), 
                            password = os.environ.get("PG_PASSWORD"), 
                            host=os.environ.get("PG_HOST"), port=os.environ.get("PG_PORT"))
        print("Opened database successfully\n")
        return conn
    except psycopg2.Error as e:
        print("I am unable to connect to the database\n")
        print(e)
        print(e.pgcode)
        print(e.pgerror)
        print(traceback.format_exc())
        return None

def get_query_string(table_name, geometry_column, epsg_code):
    '''Build query string from parameter inputs defining table name, geometry column
    name, and epsg code.
    '''
    query_string = 'SELECT ST_TRANSFORM({}, {}) AS geometry FROM {}'.format(
            geometry_column, epsg_code, table_name
    )
    return query_string
    
def get_geometry_from_postgis(postgis_params):
    '''
    This function takes a dictionary containing parameters for building a SQL query,
    as defined in get_query_string, then connects to a postgis db, selects the 
    data specified in the query, and finally returns a geodataframe with a single
    column named geometry that contains shape data.
    '''
    query_string = get_query_string(**postgis_params['query'])
    conn = pgconnect()
    #cur = conn.cursor()
    print("SQL QUERY = "+query_string+'\r\n')
    try:
        geo_df = gpd.GeoDataFrame.from_postgis(
            query_string, 
            conn, 
            geom_col='geometry', 
            crs={'init': u'epsg:{}'.format(postgis_params['query']['epsg_code'])}, 
            coerce_float=False
        )
        return geo_df
    except Exception as e:
        print(e)
    finally:
        conn.close()
        
def get_geometry_from_file(name, geom_col='geometry'):
    """Import geometry from a file using geopandas.read_file
    Returns only the geometry column!
    """
    gdf = gpd.read_file(name)
    #print(type(gdf))
    return gdf

def get_geodf_geometry(**kwargs):
    print(kwargs)
    """Import geometry either from file or from postgis db"""
    if 'from_file' in kwargs:
        return get_geometry_from_file(**kwargs['from_file'])
    elif 'from_postgis' in kwargs:
        return get_geometry_from_postgis(kwargs['from_postgis'])

In [4]:
# Functions for manipulating geoshapes and raster files
def get_gdfcrs_epsg(gdf):
    """Return integer EPSG code corresponding to Coordinate Reference
    used in input geodataframe, gdf. Attribute gdf.crs must contain
    a dict with key = 'init' that contains a string starting with 'epsg',
    followed by a colon, followed by an integer as a string.
    """
    try:
        dfepsg = gdf.crs['init'].split(':')
        if dfepsg[0] == 'epsg':
            return int(dfepsg[1])
    except:
        print('Error: geodataframe crs = {}, unrecognized EPSG integer'.format(gdf.crs['init']))

def get_raster_info_srs(raster_file, print_info=True):
    """Print information about raster file, and return its
    spatial reference system using gdal.
    """
    if print_info:
        try:
            print(gdal.Info(raster_file))
        except:
            print("Error reading info from raster file = {}".format(raster_file))
    try:
        raster = gdal.Open(raster_file)
    except:
        print("Error opening raster file = {}".format(raster_file))
    else:
        raster_srs = osr.SpatialReference()
        raster_srs.ImportFromWkt(raster.GetProjection())
        return raster_srs

def transform_polygons_from_srsinp_to_srsout(geom_col, srs_out):
    """Transform list of georeferenced polygon geometries from geopandas
    dataframe geometry column, geom_col, to the desired output Spatial Reference, srs_out.
    The input geometry column, geom_col, MUST have a valid epsg code defining its Spatial 
    Reference (SRS)."""
    # Define input spatial reference using epsg code from gdf
    srs_inp = ogr.osr.SpatialReference()
    srs_inp.ImportFromEPSG(get_gdfcrs_epsg(geom_col))
    poly_out = []
    # Define list of polygons in transformed spatial reference
    for g in geom_col:
        # If MultiPolygon, then assume it contains only one Polygon
        if g.type == 'MultiPolygon':
            poly = ogr.CreateGeometryFromWkt(g.geoms[0].wkt)
        elif g.type == 'Polygon':
            poly = ogr.CreateGeometryFromWkt(g.wkt)
        else:
            print("Error: geometry = {}, MUST be Polygon or MultiPolygon".format(g.type))
        poly.AssignSpatialReference(srs_inp)
        # Transform point co-ordinates so that they are in same projection as raster 
        poly.Transform(osr.CoordinateTransformation(srs_inp, srs_out))
        poly_out.append(poly.ExportToWkt())
    return poly_out

In [5]:
# Functions for computing raster statistics
def get_raster_stats_df(geom_ras, df_index, raster_file, raster_name, **kwargs):
    """Compute raster statistics for input geometry and raster file.
    Return results in dataframe
    """
    geomstats = zonal_stats(geom_ras, raster_file, **kwargs)
    df_gs = pd.DataFrame(geomstats, index=df_index)
    df_gs.rename(columns={co: raster_name+'_'+co for co in df_gs.columns}, inplace=True)
    return df_gs

# Functions for classifying raster statistics
def get_stats_classification(gdf, **kwargs):
    """Classify raster statistics using specified parameters in kwargs"""
    raster_names = [rn for rn in kwargs.keys() if rn != "stats_to_class"]
    for rn in raster_names:
        stats_to_class = [rn+'_'+sc for sc in kwargs['stats_to_class']]
        levels = kwargs[rn]['levels']
        labels = kwargs[rn]['level_labels']
        ctag = '_' + kwargs[rn]['class_tag']
        for s2c in stats_to_class:
            try:
                gdf[s2c+ctag] = pd.cut(gdf[s2c], levels, right=True, labels=labels)
            except KeyError:
                print("Key Error exception occurred for raster stat key = {}".format(s2c))
            except Exception as e:
                print("A non-key error exception occurred: {}".format(e))
    return gdf

In [6]:
#help(gpd.read_file)

In [7]:
# Step 1) Select geometry column either from Postgis db (implemented), or
# from shapefile (not yet implemented).  In eithe case, make sure geometry
# has a valid epsg Spatial reference assigned to it, such as 4326 (lon/lat)
# For a Postgis-derived geometry, this is done on the db-side using ST_TRANSFORM
# Note a copy of the original geopandas dataframe is made to preserve the dataframe
# obtained from postgis for debugging :-)
gdf = get_geodf_geometry(**params['geometry'])
gdf_merge = gdf.copy()
type(gdf_merge)
#gdf.info()
gdf_merge.info()
print(gdf['geometry'][0].type)
print(gdf['geometry'].crs)

#gdf = get_geometry_from_file("./Data/DisasterNeighborhoods/RLIS_ST_clips_pdx_final.dbf")
#gdf_shape = gpd.read_file("./Data/DisasterNeighborhoods/RLIS_ST_clips_pdx_final.dbf")
#gdf_shape.info()
#print(gdf_shape['geometry'][0].type)
#print(gdf_shape['geometry'].crs)

{'from_file': {'name': './Data/DisasterNeighborhoods/RLIS_ST_clips_pdx_final.dbf', 'geom_col': 'geometry'}}
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 5 columns):
fid_1       128 non-null int64
name        128 non-null object
area        128 non-null float64
sqmile      128 non-null float64
geometry    128 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 5.1+ KB
Polygon
{'init': 'epsg:4326'}


In [8]:
# Steps 2 through 5 are repeated for each raster file
# the results for each raster are appended to gdf_merge
for raster_name in params['raster']['names']:
    # Step 2) Print info about tif file (optional) and get its spatial reference info
    raster_file = params['raster']['root'] + raster_name + params['raster']['ext']
    print("Computing statistics for raster file = {}".format(raster_file))
    srs_raster = get_raster_info_srs(raster_file, print_info=False)

    # Step 3) Generate a list of polygons transformed from the srs used in the 
    # input geodataframe, gdf, to the srs used in the raster file, srs_raster
    geom_ras = transform_polygons_from_srsinp_to_srsout(gdf_merge['geometry'], srs_raster)
    #print("# of rows in transformed geometry ({}) and geodataframe ({}) should be equal!".format(
    #        len(geom_ras), len(gdf)
    #))

    # Step 4) Use rasterstats to compute analytics on pixel values within specified geometry, 
    # MUST be polygon or multipolygon and transformed to srs_raster!
    # Add stats from pixel values into geodataframe that defines geometry
    df_gs = get_raster_stats_df(geom_ras, gdf_merge.index, raster_file, raster_name, 
                                **params['zonal_stats']
    )
    
    # Step 5) Aggregate the statistics from each raster into a final merged geodataframe
    gdf_merge = gdf_merge.join(df_gs)

Computing statistics for raster file = ./CSZ_M9p0_pgv_site.tif


/home/jupyter/.conda/envs/jupyter/lib/python3.6/site-packages/rasterstats/main.py:145: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  with Raster(raster, affine, nodata, band) as rast:
/home/jupyter/.conda/envs/jupyter/lib/python3.6/site-packages/rasterstats/io.py:242: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  self.affine = guard_transform(self.src.transform)
/home/jupyter/.conda/envs/jupyter/lib/python3.6/site-packages/rasterstats/main.py:165: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  np.issubdtype(fsrc.array.dtype, float)


Computing statistics for raster file = ./CSZ_M9p0_PGD_landslide_dry.tif


/home/jupyter/.conda/envs/jupyter/lib/python3.6/site-packages/rasterstats/io.py:294: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


Computing statistics for raster file = ./CSZ_M9p0_PGD_landslide_wet.tif
Computing statistics for raster file = ./CSZ_M9p0_PGD_liquefaction_wet.tif


In [9]:
gdf_merge.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 25 columns):
fid_1                         128 non-null int64
name                          128 non-null object
area                          128 non-null float64
sqmile                        128 non-null float64
geometry                      128 non-null object
pgv_site_count                128 non-null int64
pgv_site_max                  109 non-null float64
pgv_site_mean                 109 non-null float64
pgv_site_min                  109 non-null float64
pgv_site_std                  109 non-null float64
PGD_landslide_dry_count       128 non-null int64
PGD_landslide_dry_max         120 non-null float64
PGD_landslide_dry_mean        120 non-null float64
PGD_landslide_dry_min         120 non-null float64
PGD_landslide_dry_std         120 non-null float64
PGD_landslide_wet_count       128 non-null int64
PGD_landslide_wet_max         119 non-null float64
PGD_landslide_wet_mean        

In [10]:
# Step 6) Classify a subset of the geometry statistics, converting the calculated
# stat in pixel values to a label describing the stats intensity bin
if 'stats_classification' in params:
    gdf_merge_class = get_stats_classification(gdf_merge, **params['stats_classification'])
gdf_merge_class.info()
print(gdf_merge_class[['geometry', 'pgv_site_count', 'pgv_site_max', 'pgv_site_max_MMI']])

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 37 columns):
fid_1                           128 non-null int64
name                            128 non-null object
area                            128 non-null float64
sqmile                          128 non-null float64
geometry                        128 non-null object
pgv_site_count                  128 non-null int64
pgv_site_max                    109 non-null float64
pgv_site_mean                   109 non-null float64
pgv_site_min                    109 non-null float64
pgv_site_std                    109 non-null float64
PGD_landslide_dry_count         128 non-null int64
PGD_landslide_dry_max           120 non-null float64
PGD_landslide_dry_mean          120 non-null float64
PGD_landslide_dry_min           120 non-null float64
PGD_landslide_dry_std           120 non-null float64
PGD_landslide_wet_count         128 non-null int64
PGD_landslide_wet_max           119 non-null floa

In [ ]:
# Write results to csv file
if 'write_csv' in params:
    gdf_merge_class.to_csv(params['write_csv']['name'])

### Plotting

In [ ]:
# Examine the disaster neighborhood geometries with very small count (# of pixel) values
# a count of zero leads to NaNs for the stats, so need to investigate these further
# The histograms show an abundance of very small geometries (count < 50)
# Investigate rows with counts below 50?, what is too small and should be "cleaned up"?
plt.subplot(2, 1, 1)
plt.hist(gdf_merge_class['pgv_site_count'])
plt.subplot(2, 1, 2)
plt.hist(gdf_merge_class['pgv_site_count'], bins=[0, 20, 50, 200, 500, 1000])
plt.xlim([0, 1000])
print(gdf_merge_class[gdf_merge_class['pgv_site_count']<50])
gdf_merge_class[gdf_merge_class['pgv_site_count']<50].to_csv(\
        "DisasterNeighborhoods_with_small_counts.csv"
)

In [ ]:
# Make a map showing distribution of df column, col2plot_y
#col2plot_y = 'casualties_avetotal'
#col2plot_y = 'buildingloss'
#col2plot_y = 'buildingloss_ratio'
col2plot_y = 'nupv_mean'

gplt.choropleth(gdf_merge,
                hue=gdf_merge[col2plot_y],  # Display data, passed as a Series
                projection=gplt.crs.AlbersEqualArea(),
                cmap='Purples',
                linewidth=0.5,
                edgecolor='black',
                k=5,
                legend=True,
                scheme='equal_interval',
                figsize=(12, 12)
)
plt.title("{}".format(col2plot_y), fontsize=24)